In [1]:
from ydc.features.get_features import get_features
import pandas as pd
import numpy as np

from pprint import pprint

In [2]:
(features, df, box, combos, cells, n_ind, n_dist) = get_features(status=True)

In [3]:
def _error(df_feat, quality, idx, coeffs):
    if idx is not None:
        return (quality[idx] - (df_feat[idx] * coeffs).sum(axis=1)).abs().mean()
    else:
        return (quality - (df_feat * coeffs).sum(axis=1)).abs().mean()
    
def principal_values(A, n):
    U, s, Vt = np.linalg.svd(A, full_matrices=False)
    V = Vt.T

    # sort the PCs by descending order of the singular values (i.e. by the
    # proportion of total variance they explain)
    ind = np.argsort(s)[::-1]
    U = U[:, ind]
    s = s[ind]
    V = V[:, ind]

    S = np.diag(s)
    # if we use only the first 20 PCs the reconstruction is less accurate
    Ahat = np.dot(U[:, :n], np.dot(S[:n, :n], V[:,:n].T))
    print("Using first %s PCs, MSE = %.6G" %(n, np.mean((A - Ahat)**2)))
    
    return Ahat

def get_coeffs_by_cat(df_feat, df_busi, combos):
    coeffs = {}
    errors = {}
    
    for combo in combos:
        idx = (df_busi['category'] == combo)
        
        c = get_coeffs(df_feat[idx], df_busi.loc[idx, 'real_stars'])
        coeffs[combo] = c
        
        errors[combo] = {
            "all": _error(df_feat, df_busi['real_stars'], None, c),
            "combo": _error(df_feat, df_busi['real_stars'], idx, c),
            "other": _error(df_feat, df_busi['real_stars'], -idx, c)
        }

    return coeffs, errors
    
def get_coeffs(df_feat, quality):
    if (df_feat.empty):
        return pd.Series(0, index=df_feat.columns)
    
    a = df_feat.values
    b = quality.values
    lstsq_res = np.linalg.lstsq(a,b)[0]
    
    return pd.Series(lstsq_res, index=df_feat.columns)

def find_best_location():
    return


In [5]:
# Compare with just a constant prediction
error = abs(df['real_stars'] - df['real_stars'].mean())
print("Guessing a constant %f" % df['real_stars'].mean())
print("error-mean: %f" % error.mean())
print("error-std:  %f" % error.std())

Guessing a constant 3.659542
error-mean: 0.733059
error-std:  0.547270


In [6]:
c_all = get_coeffs(features,df['real_stars'])
e_all = _error(features, df['real_stars'], None, c_all)
print(e_all)

0.718404888177


In [26]:
from sklearn import linear_model

In [31]:
mdl = linear_model.Ridge(alpha=.1)

A = features.values
b = df['real_stars'].values
mdl.fit(A, b)

error = abs(mdl.predict(A) - b)
print("error-mean: %f" % error.mean())
print("error-std:  %f" % error.std())

error-mean: 0.718405
error-std:  0.539900


In [28]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
model = Pipeline([('poly', PolynomialFeatures(degree=2)),
                  ('linear', Ridge(alpha=.1))])

In [29]:
A = features.values
b = df['real_stars'].values
mdl = model.fit(A, b)

In [31]:
error = abs(mdl.predict(A) - b)
print("error-mean: %f" % error.mean())
print("error-std:  %f" % error.std())

error-mean: 0.698239
error-std:  0.530030


In [3]:
features.columns

Index(['weighted review-count', 'reviews_mean', 'reviews_median',
       'reviews_std', 'reviews_max', 'reviews_min', 'reviews_sum',
       'neighbourhood_radius', 'stars_mean', 'stars_median', 'stars_std',
       'stars_max', 'stars_min', 'stars_sum'],
      dtype='object')

In [13]:
ret = n_dist.apply(np.max)
ret.name = 'neighbourhood_rrawr'

In [15]:
from ydc.tools.import_data import import_reviews

In [17]:
reviews = import_reviews(
        fields=['business_id', 'stars', 'date', 'real_date'])

In [18]:
reviews.groupby('business_id')